<a href="https://colab.research.google.com/github/ansfarooq7/l4-project/blob/main/protoypes/L4_Project_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Code from https://ramsrigoutham.medium.com/sized-fill-in-the-blank-or-multi-mask-filling-with-roberta-and-huggingface-transformers-58eb9e7fb0c

In [48]:
!pip install transformers
import torch

In [49]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    #random.seed(seed)
    #np.random.seed(seed)
    #if is_torch_available():
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    #if is_tf_available():
        #tf.random.set_seed(seed)
    
def get_prediction(sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
        print("Mask ",index+1,"Guesses : ",words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess

sentence = "Manchester United are ___ ___ ___ champions."
print ("Original Sentence: ",sentence)
sentence = sentence.replace("___","<mask>")
print ("Original Sentence replaced with mask: ",sentence)
print ("\n")

predicted_blanks = get_prediction(sentence)
print ("\nBest guess for fill in the blank :::",predicted_blanks)

Original Sentence:  Manchester United are ___ ___ ___ champions.
Original Sentence replaced with mask:  Manchester United are <mask> <mask> <mask> champions.


Mask  1 Guesses :  ['the', 'defending', 'currently', 'reigning', 'crowned']
Mask  2 Guesses :  ['reigning', 'defending', 'the', 'crowned', 'Premier']
Mask  3 Guesses :  ['League', 'league', 'defending', 'world', 'four']

Best guess for fill in the blank :::  the reigning League


In [50]:
from transformers import pipeline
text_generation = pipeline("text-generation")

limericks = []
set_seed(31)

starting_words = [["That", "Had", "Not", "But", "That"], 
                  ["There", "Who", "She", "Tormenting", "Til"],
                  ["Relentless", "This", "First", "and", "then"],
                  ["There", "Who", "That", "To", "She"],
                  ["There", "Who", "Two", "Four", "Have"]]

rhyming_words = [["told", "bold", "woodchuck", "truck", "road"], 
                 ["Nice", "grease", "house", "spouse", "peace"],
                 ["deadlines", "lines", "edits", "credits", "wine"],
                 ["Lynn", "thin", "essayed", "lemonade", "in"],
                 ["beard", "feared", "hen", "wren", "beard"]]

for i in range(len(starting_words)):
  limerick = ""

  for j in range(5):
    gpt2_sentence = text_generation(starting_words[i][j], max_length=3, do_sample=False)[0]
    sentence = gpt2_sentence['generated_text'] + " ___ ___ ___ " + rhyming_words[i][j]
    print("Original Sentence: ",sentence)
    sentence = sentence.replace("___","<mask>")
    print("Original Sentence replaced with mask: ",sentence)
    print("\n")

    predicted_blanks = get_prediction(sentence)
    print("\nBest guess for fill in the blank: ", predicted_blanks)
    limerick = limerick + gpt2_sentence['generated_text'] + predicted_blanks + " " + rhyming_words[i][j] + "\n"

  limericks.append(limerick)

print("\n")
print(f"Generated {len(limericks)} limericks: \n")
for limerick in limericks:
  print(limerick)

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original Sentence:  That is why ___ ___ ___ told
Original Sentence replaced with mask:  That is why <mask> <mask> <mask> told


Mask  1 Guesses :  ['you', 'the', 'we', 'I', 'they']
Mask  2 Guesses :  ['will', 'should', 'must', 'is', 'are']
Mask  3 Guesses :  ['be', 'been', 'not', 'never', 'being']

Best guess for fill in the blank:   you will be


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original Sentence:  Had the same ___ ___ ___ bold
Original Sentence replaced with mask:  Had the same <mask> <mask> <mask> bold


Mask  1 Guesses :  ['font', 'color', 'name', 'text', 'letters']
Mask  2 Guesses :  [',', 'but', '.', 'color', 'and']
Mask  3 Guesses :  ['in', ':', 'but', 'be', ',']

Best guess for fill in the blank:   font , in


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Original Sentence:  Who is the ___ ___ ___ feared
Original Sentence replaced with mask:  Who is the <mask> <mask> <mask> feared


Mask  1 Guesses :  ['person', 'man', 'terrorist', 'leader', 'one']
Mask  2 Guesses :  ['who', 'leader', 'and', ',', 'that']
Mask  3 Guesses :  ['is', 'most', 'and', 'be', 'are']

Best guess for fill in the blank:   person who is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original Sentence:  Two, the ___ ___ ___ hen
Original Sentence replaced with mask:  Two, the <mask> <mask> <mask> hen


Mask  1 Guesses :  ['mother', 'm', 'f', 'hen', 'n']
Mask  2 Guesses :  ['of', '-', 'and', "'s", ',']
Mask  3 Guesses :  ['the', 'a', 'mother', 'he', 'house']

Best guess for fill in the blank:   mother of the


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original Sentence:  Four, the ___ ___ ___ wren
Original Sentence replaced with mask:  Four, the <mask> <mask> <mask> wren


Mask  1 Guesses :  ['m', 'great', 'k', 'last', 'n']
Mask  2 Guesses :  ['of', '-', 'and', ',', 'th']
Mask  3 Guesses :  ['the', 'of', 'a', 'ian', "'s"]

Best guess for fill in the blank:   m of the
Original Sentence:  Have the same ___ ___ ___ beard
Original Sentence replaced with mask:  Have the same <mask> <mask> <mask> beard


Mask  1 Guesses :  ['dark', 'hair', 'full', 'long', 'gray']
Mask  2 Guesses :  ['-', 'and', 'as', 'but', ',']
Mask  3 Guesses :  ['your', 'and', 'a', 'no', 'white']

Best guess for fill in the blank:   dark - your


Generated 5 limericks: 

That is why you will be told
Had the same font , in bold
Not the only one - the woodchuck
But the most important is the truck
That is why I built the road

There is no way to in Nice
Who is the master of elbow grease
She, who lived in the house
Tormenting the life of the spouse
Tilted by , of peace

Re